In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time, os, copy, argparse
import multiprocessing
from torchsummary import summary
from matplotlib import pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [ ]:
train_dir = 'three_data/train'
val_dir = 'three_data/val'
test_dir = 'three_data/test'
one_test_dir = 'three_data/one_test'
one_val_dir = 'three_data/one_val'

num_epochs = 1

batch_size = 1

num_classes = 4

num_cpu = multiprocessing.cpu_count()

In [ ]:

path_base = 'three_data/train'
img_path = path_base + '/peaberry/1001.jpg'
img = Image.open(img_path)
img

img_np = np.array(img)


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor() 
])
img_tr = transform(img)

img_np = np.array(img_tr)

In [ ]:
mean, std = img_tr.mean([1,2]), img_tr.std([1,2])
print("正規化前の平均値（mean）と標準偏差（std）:")
print("画像の平均値（mean）      :", mean)
print("画像の標準偏差（std）:", std)


In [ ]:
transform_norm = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean, std)  
])
img_normalized = transform_norm(img)
img_np = np.array(img_normalized)

In [ ]:
img_normalized = transform_norm(img)
img_normalized = np.array(img_normalized)
img_normalized = img_normalized.transpose(1, 2, 0)

plt.imshow(img_normalized)
plt.xticks([]) 
plt.yticks([]) 


In [ ]:
img_nor = transform_norm(img)
mean, std = img_nor.mean([1,2]), img_nor.std([1,2])
print("正規化された画像の平均値と標準偏差:")
print("画像の平均値        :", mean)
print("画像の標準偏差      :", std)


In [ ]:
# 画像の正規化に使用する平均値（mean）と標準偏差（std）の変数を作成する
mean = [0.8609, 0.8271, 0.7862]
std = [0.1960, 0.2179, 0.2592]


In [ ]:
image_transforms = {
    'train': transforms.Compose([
        transforms.Resize(size=(256, 256)), 
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(),  
        transforms.Normalize(mean, std) 
    ]),
    'val': transforms.Compose([
        transforms.Resize(size=(256, 256)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean, std)  
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=(256, 256)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean, std)  
    ]),
    'one_test': transforms.Compose([
        transforms.Resize(size=(256, 256)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean, std)  
    ]),
    'one_val': transforms.Compose([
        transforms.Resize(size=(256, 256)),  
        transforms.ToTensor(),  
        transforms.Normalize(mean, std)  
    ]),
}


In [ ]:
import shutil
import os

for directory in [train_dir, val_dir, test_dir]:
    checkpoints_path = os.path.join(directory, '.ipynb_checkpoints')
    if os.path.exists(checkpoints_path):
        shutil.rmtree(checkpoints_path)


In [ ]:
dataset = {
    'train': datasets.ImageFolder(root=train_dir, transform=image_transforms['train']),
    'val': datasets.ImageFolder(root=val_dir, transform=image_transforms['val']),
    'test': datasets.ImageFolder(root=test_dir, transform=image_transforms['test']),
    'one_test': datasets.ImageFolder(root=one_test_dir, transform=image_transforms['one_test']),
    'one_val': datasets.ImageFolder(root=one_val_dir, transform=image_transforms['one_val']),
}


In [ ]:
classes = dataset['train'].classes
print('このデータセットには', len(classes), '個のクラスがあり、クラスは次の通りです:', classes)


In [ ]:
dataset_sizes = {
    'train': len(dataset['train']),
    'val': len(dataset['val']),
    'test': len(dataset['test']),
    'one_val': len(dataset['one_val']),
    'one_test': len(dataset['one_test']),
}


In [ ]:
print("トレーニングデータの画像数   :", dataset_sizes['train'])
print("検証データの画像数           :", dataset_sizes['val'])
print("テストデータの画像数         :", dataset_sizes['test'])
print("テストデータの画像数         :", dataset_sizes['one_test'])
print("テストデータの画像数         :", dataset_sizes['one_val'])


In [ ]:
dataloaders = {
    'train': data.DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True),
    'val'  : data.DataLoader(dataset['val'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True),
    'test' : data.DataLoader(dataset['test'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True),
    'one_test' : data.DataLoader(dataset['one_test'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True),
    'one_val' : data.DataLoader(dataset['one_val'], batch_size=batch_size, shuffle=True, num_workers=num_cpu, pin_memory=True, drop_last=True)

}

In [ ]:
def imshow(inp, title=None, mean=[0.8782, 0.8429, 0.8053], std=[0.1224, 0.1398, 0.1678]):
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean 
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

inputs, classes = next(iter(dataloaders['train']))
class_names = ['defect', 'longberry', 'peaberry', 'premium']
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])


In [ ]:
print(dataset['train'].class_to_idx)
print(dataset['val'].class_to_idx)
print(dataset['test'].class_to_idx)
print(dataset['one_test'].class_to_idx)
print(dataset['one_val'].class_to_idx)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
images, labels = images.to(device), labels.to(device)


In [ ]:
print("\nEfficientNetV2を読み込んでいます \n")
model_ft = models.efficientnet_v2_s(pretrained=True)
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)
model_ft = model_ft.to(device)
for num, (name, param) in enumerate(model_ft.named_parameters()):
    print(num, name, param.requires_grad)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [ ]:
print('モデルの概要:\n')
summary(model_ft, input_size=(3, 256, 256))
print(model_ft)


In [ ]:
writer=SummaryWriter('/content/content/runs/summary_1')
writer.add_graph(model_ft, images)

In [ ]:
unique_labels = torch.unique(labels)
print(f"Unique labels: {unique_labels}")


In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from tqdm import tqdm
def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    writer = SummaryWriter("content/runs/model_percobaan_1")

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device, non_blocking=False)
                labels = labels.to(device, non_blocking=False)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'train':
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
            else:
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    writer.close()
    return model

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=num_epochs)


In [ ]:
!kill 5593

In [ ]:
%load_ext tensorboard
%tensorboard --logdir content/runs
%reload_ext tensorboard


In [ ]:
from tensorboard import notebook
notebook.list() 


In [ ]:
PATH = 'model_percobaan_1.pth'
print("\nモデルを保存しています...")
torch.save(model_ft, PATH)


In [ ]:
import torch

PATH = 'model_percobaan_1.pth'
print("\nモデルを復元しています...")
model_ft = torch.load(PATH)

model_ft.eval()


In [ ]:
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images // 2, 2, images_so_far)
                ax.axis('off')

                if preds[j] < len(class_names):
                    ax.set_title(f'yosoku: {class_names[preds[j]]}')
                else:
                    ax.set_title('yosoku: humei')

                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)
import torch

def computeTestSetAccuracy(model, criterion):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad():
        model.eval()

        for j, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            if labels.ndim > 1: 
                labels = labels.argmax(dim=1)

            outputs = model_ft(inputs)

            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)

            _, predictions = torch.max(outputs, 1)
            correct_counts = predictions.eq(labels)
            acc = correct_counts.sum().float() / labels.size(0)

            test_acc += acc.item() * inputs.size(0)

            print("テストバッチ番号: {:03d}, テスト損失: {:.4f}, 精度: {:.4f}".format(j, loss.item(), acc.item()))

    avg_test_loss = test_loss / dataset_sizes['test']
    avg_test_acc = test_acc / dataset_sizes['test']

    print("\nテストセット: 平均損失: {:.4f}, 平均精度: {:.4f}".format(avg_test_loss, avg_test_acc))
    return avg_test_loss, avg_test_acc

def plot_outputs_without_reduction_and_save_csv(model, dataloader, csv_filename="outputs_with_labels.csv"):
    model.eval()
    outputs_list = []
    true_labels_list = []
    predicted_labels_list = []

    with torch.no_grad():
        for inputs, true_labels in dataloader:
            inputs = inputs.to(device)
            true_labels = true_labels.to(device)

            outputs = model(inputs)
            outputs_list.append(outputs.cpu().numpy())
            true_labels_list.append(true_labels.cpu().numpy())

            _, predicted_labels = torch.max(outputs, 1)
            predicted_labels_list.append(predicted_labels.cpu().numpy())

    outputs_all = np.concatenate(outputs_list, axis=0)
    true_labels_all = np.concatenate(true_labels_list, axis=0)

    if outputs_all.shape[1] >= 3:
        outputs_3d = outputs_all[:, :3]
    else:
        raise ValueError("モデル出力の次元が3未満です。3次元以上の出力が必要です。")

    data_to_save = np.column_stack((outputs_3d, true_labels_all, predicted_labels_all))
    df = pd.DataFrame(data_to_save, columns=["Output1", "Output2", "Output3", "True Label", "Predicted Label"])
    df.to_csv(csv_filename, index=False)
    print(f"データが{csv_filename}に保存されました。")

    centers = {}
    for label in np.unique(true_labels_all):
        centers[label] = outputs_3d[true_labels_all == label].mean(axis=0)

    fig = go.Figure()

    for i in range(len(np.unique(true_labels_all))):
        idx = true_labels_all == i
        fig.add_trace(go.Scatter3d(
            x=outputs_3d[idx, 0],
            y=outputs_3d[idx, 1],
            z=outputs_3d[idx, 2],
            mode='markers',
            marker=dict(size=2),  
            name=f'Class {i}'
        ))

    for label, center in centers.items():
        fig.add_trace(go.Scatter3d(
            x=[0, center[0]],
            y=[0, center[1]],
            z=[0, center[2]],
            mode='lines',
            line=dict(width=4),
            name=f'Class {label} Center Line'
        ))

    fig.update_layout(scene=dict(
        xaxis_title="Output Dimension 1",
        yaxis_title="Output Dimension 2",
        zaxis_title="Output Dimension 3"
    ), title="3D Output Space with Center Lines to Origin")

    fig.show()
    fig.write_html("3D_plot_without_reduction.html")

plot_outputs_without_reduction_and_save_csv(model_ft, dataloaders['test'])


In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['one_val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images // 2, 2, images_so_far)
                ax.axis('off')

                if preds[j] < len(class_names):
                    ax.set_title(f'yosoku: {class_names[preds[j]]}')
                else:
                    ax.set_title('yosoku: humei')  
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)
        

import torch

def computeTestSetAccuracy(model, criterion):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad():

        model.eval()

        for j, (inputs, labels) in enumerate(dataloaders['one_test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            if labels.ndim > 1: 
                labels = labels.argmax(dim=1)

            outputs = model_ft(inputs)

            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)

            _, predictions = torch.max(outputs, 1)
            correct_counts = predictions.eq(labels)
            acc = correct_counts.sum().float() / labels.size(0)

            test_acc += acc.item() * inputs.size(0)

            print("テストバッチ番号: {:03d}, テスト損失: {:.4f}, 精度: {:.4f}".format(j, loss.item(), acc.item()))

    avg_test_loss = test_loss / dataset_sizes['one_test']
    avg_test_acc = test_acc / dataset_sizes['one_test']

    print("\nテストセット: 平均損失: {:.4f}, 平均精度: {:.4f}".format(avg_test_loss, avg_test_acc))
    return avg_test_loss, avg_test_acc

idx_to_class = {v: k for k, v in dataset['one_test'].class_to_idx.items()}
print(idx_to_class)

import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def plot_outputs_without_reduction_and_save_csv(model, dataloader, csv_filename="soutputs_with_labels.csv"):
    model.eval()
    outputs_list = []
    true_labels_list = []
    predicted_labels_list = []

    with torch.no_grad():
        for inputs, true_labels in dataloader:
            inputs = inputs.to(device)
            true_labels = true_labels.to(device)

            outputs = model(inputs)
            outputs_list.append(outputs.cpu().numpy())
            true_labels_list.append(true_labels.cpu().numpy())

            _, predicted_labels = torch.max(outputs, 1)
            predicted_labels_list.append(predicted_labels.cpu().numpy())

    outputs_all = np.concatenate(outputs_list, axis=0)
    true_labels_all = np.concatenate(true_labels_list, axis=0)
    predicted_labels_all = np.concatenate(predicted_labels_list, axis=0)

    if outputs_all.shape[1] >= 3:
        outputs_3d = outputs_all[:, :3]
    else:
        raise ValueError("モデル出力の次元が3未満です。3次元以上の出力が必要です。")

    data_to_save = np.column_stack((outputs_3d, true_labels_all, predicted_labels_all))
    df = pd.DataFrame(data_to_save, columns=["Output1", "Output2", "Output3", "True Label", "Predicted Label"])
    df.to_csv(csv_filename, index=False)
    print(f"データが{csv_filename}に保存されました。")

    centers = {}
    for label in np.unique(true_labels_all):
        centers[label] = outputs_3d[true_labels_all == label].mean(axis=0)

    fig = go.Figure()

    for i in range(len(np.unique(true_labels_all))):
        idx = true_labels_all == i
        fig.add_trace(go.Scatter3d(
            x=outputs_3d[idx, 0],
            y=outputs_3d[idx, 1],
            z=outputs_3d[idx, 2],
            mode='markers',
            marker=dict(size=2),
            name=f'Class {i}'
        ))

    for label, center in centers.items():
        fig.add_trace(go.Scatter3d(
            x=[0, center[0]],
            y=[0, center[1]],
            z=[0, center[2]],
            mode='lines',
            line=dict(width=4),
            name=f'Class {label} Center Line'
        ))

    fig.update_layout(scene=dict(
        xaxis_title="Output Dimension 1",
        yaxis_title="Output Dimension 2",
        zaxis_title="Output Dimension 3"
    ), title="3D Output Space with Center Lines to Origin")

    fig.show()
    fig.write_html("3D_plot_without_reduction.html")

plot_outputs_without_reduction_and_save_csv(model_ft, dataloaders['one_test'])


In [ ]:
import torch
import itertools
import numpy as np

num_classes = 4  
confusion_matrix = torch.zeros(num_classes, num_classes)
model = model_ft

with torch.no_grad():
    for inputs, classes in dataloaders['test']:
        inputs = inputs.to(device)
        classes = classes.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        for t, p in zip(classes.view(-1), preds.view(-1)):
            if t < num_classes and p < num_classes:
                confusion_matrix[t.long(), p.long()] += 1


print("Confusion Matrix:")
print(confusion_matrix)


In [ ]:
ssh -L localhost:8989:192.168.11.10:8989 ratchet_VPNdef plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
  
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(cm           = np.array([[1.2000e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.1990e+03, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.0000e+00, 1.1990e+03, 0.0000e+00],
        [1.9100e+02, 2.3500e+02, 7.6700e+02, 7.0000e+00]]),
                      normalize    = True,
                      target_names = [ 'longberry', 'peaberry', 'premium','deffect'],
                      title        = "Confusion Matrix")

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook"


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.svm import OneClassSVM

data = pd.read_csv('soutputs_with_labels.csv')

X = data[['Output1', 'Output2', 'Output3']].values
true_labels = data['True Label'].values  # 真のラベル

good_data = data[data['True Label'].isin([0, 1, 2])]
X_good = good_data[['Output1', 'Output2', 'Output3']].values

bad_data = data[data['True Label'] == 3]
X_bad = bad_data[['Output1', 'Output2', 'Output3']].values

oc_svm = OneClassSVM(kernel='rbf', gamma=1, nu=0.02)
oc_svm.fit(X_bad)
oc_svm.fit(X_good)

y_pred = oc_svm.predict(X) 

good_points = X[y_pred == 1]
defective_points = X[y_pred == -1]

x = np.linspace(X[:, 0].min(), X[:, 0].max(), 30)
y = np.linspace(X[:, 1].min(), X[:, 1].max(), 30)
z = np.linspace(X[:, 2].min(), X[:, 2].max(), 30)
xx, yy, zz = np.meshgrid(x, y, z)
grid = np.c_[xx.ravel(), yy.ravel(), zz.ravel()]
decision_scores = oc_svm.decision_function(grid)
decision_scores = decision_scores.reshape(xx.shape)

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=good_points[:, 0],
    y=good_points[:, 1],
    z=good_points[:, 2],
    mode='markers',
    marker=dict(size=2, color='blue', opacity=0.8),
    name='Good (Predicted)'
))

fig.add_trace(go.Scatter3d(
    x=defective_points[:, 0],
    y=defective_points[:, 1],
    z=defective_points[:, 2],
    mode='markers',
    marker=dict(size=2, color='red', opacity=0.8),
    name='Defective (Predicted)'
))

fig.add_trace(go.Isosurface(
    x=grid[:, 0],
    y=grid[:, 1],
    z=grid[:, 2],
    value=decision_scores.ravel(),
    isomin=-0.01, 
    isomax=0.01,
    opacity=0.3,
    surface_count=1, 
    colorscale='Viridis',
    name='Decision Boundary'
))

fig.update_layout(
    title='One-Class SVM: 3D Decision Boundary with Data Points',
    scene=dict(
        xaxis_title='Output1',
        yaxis_title='Output2',
        zaxis_title='Output3',
    ),
    legend=dict(
        x=0.1, y=0.9, bgcolor='rgba(255,255,255,0.5)'
    )
)

fig.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.svm import OneClassSVM

data = pd.read_csv('outputs_with_labels.csv')

X = data[['Output1', 'Output2', 'Output3']].values
true_labels = data['True Label'].values 

good_data = data[data['True Label'].isin([0, 1, 2])]
X_good = good_data[['Output1', 'Output2', 'Output3']].values

bad_data = data[data['True Label'] == 3]
X_bad = bad_data[['Output1', 'Output2', 'Output3']].values

oc_svm = OneClassSVM(kernel='rbf', gamma=1, nu=0.02)
oc_svm.fit(X_good)

y_pred = oc_svm.predict(X) 

good_points = X[y_pred == 1]
defective_points = X[y_pred == -1]

x = np.linspace(X[:, 0].min(), X[:, 0].max(), 30)
y = np.linspace(X[:, 1].min(), X[:, 1].max(), 30)
z = np.linspace(X[:, 2].min(), X[:, 2].max(), 30)
xx, yy, zz = np.meshgrid(x, y, z)
grid = np.c_[xx.ravel(), yy.ravel(), zz.ravel()]
decision_scores = oc_svm.decision_function(grid)
decision_scores = decision_scores.reshape(xx.shape)

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=good_points[:, 0],
    y=good_points[:, 1],
    z=good_points[:, 2],
    mode='markers',
    marker=dict(size=2, color='blue', opacity=0.8),
    name='Good (Predicted)'
))

fig.add_trace(go.Scatter3d(
    x=defective_points[:, 0],
    y=defective_points[:, 1],
    z=defective_points[:, 2],
    mode='markers',
    marker=dict(size=2, color='red', opacity=0.8),
    name='Defective (Predicted)'
))

fig.add_trace(go.Isosurface(
    x=grid[:, 0],
    y=grid[:, 1],
    z=grid[:, 2],
    value=decision_scores.ravel(),
    isomin=-0.01,
    isomax=0.01,
    opacity=0.3,
    surface_count=1, 
    colorscale='Viridis',
    name='Decision Boundary'
))

fig.update_layout(
    title='One-Class SVM: 3D Decision Boundary with Data Points',
    scene=dict(
        xaxis_title='Output1',
        yaxis_title='Output2',
        zaxis_title='Output3',
    ),
    legend=dict(
        x=0.1, y=0.9, bgcolor='rgba(255,255,255,0.5)'
    )
)

fig.show()


In [ ]:
print(f"Kernel: {oc_svm.kernel}")
print(f"Gamma: {oc_svm.gamma}")
print(f"Nu: {oc_svm.nu}")

y_train_pred = oc_svm.predict(X_good) 

train_accuracy = np.mean(y_train_pred == 1)  
y_bad_pred = oc_svm.predict(X_bad) 

defective_accuracy = np.mean(y_bad_pred == -1) 
average_accuracy = (train_accuracy + defective_accuracy) / 2

print(f"Average Training Accuracy: {average_accuracy:.2%}")
print(f"Training Accuracy (Good Data): {train_accuracy:.2%}")
print(f"Training Accuracy (Defective Data): {defective_accuracy:.2%}")


In [ ]:
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
X_good_scaled = scaler.fit_transform(X_good) 

gamma_values = np.arange(0.02, 5.0, 0.02) 
nu_values = np.arange(0.02, 0.1, 0.02)

best_train_accuracy = 0
best_params = {}

for gamma in gamma_values:
    for nu in nu_values:
        try:
            oc_svm = OneClassSVM(kernel='rbf', gamma=gamma, nu=nu)
            oc_svm.fit(X_good)  

            y_train_pred = oc_svm.predict(X_good)
            train_accuracy = np.mean(y_train_pred == 1)  # 良品データの正解率

            if train_accuracy > best_train_accuracy:
                best_train_accuracy = train_accuracy
                best_params = {'gamma': gamma, 'nu': nu}

            print(f"Gamma: {gamma}, Nu: {nu}, Training Accuracy: {train_accuracy:.4f}")
        except ValueError as e:
            print(f"Error with gamma={gamma}, nu={nu}: {e}")

print("\nBest Parameters:")
print(f"Gamma: {best_params['gamma']}, Nu: {best_params['nu']}")
print(f"Best Training Accuracy: {best_train_accuracy:.4f}")


In [ ]:
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd

data = pd.read_csv('outputs_with_labels.csv')

X = data[['Output1', 'Output2', 'Output3']].values
true_labels = data['True Label'].values  
good_data = data[data['True Label'].isin([0, 1, 2])]
X_good = good_data[['Output1', 'Output2', 'Output3']].values
bad_data = data[data['True Label'] == 3]
X_bad = bad_data[['Output1', 'Output2', 'Output3']].values
sdata = pd.read_csv('soutputs_with_labels.csv')
X = sdata[['Output1', 'Output2', 'Output3']].values
strue_labels = sdata['True Label'].values 
sgood_data = sdata[sdata['True Label'].isin([0, 1, 2])]
sX_good = sgood_data[['Output1', 'Output2', 'Output3']].values  
y_good = np.ones(len(sgood_data), dtype=int) 
sbad_data = sdata[sdata['True Label'] == 3]
X_bad = sbad_data[['Output1', 'Output2', 'Output3']].values
y_bad = -1 * np.ones(len(sbad_data), dtype=int) 
X_selected = np.concatenate([sX_good, X_bad], axis=0)
y_selected = np.concatenate([y_good, y_bad], axis=0) 

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
oc_svm = OneClassSVM(kernel='rbf', gamma=0.4, nu=0.02)
oc_svm.fit(X_good) 

y_pred = oc_svm.predict(X_selected)
print(classification_report(y_selected, y_pred, target_names=['Defective', 'Good']))
print(f"Kernel: {oc_svm.kernel}")
print(f"Gamma: {oc_svm.gamma}")
print(f"Nu: {oc_svm.nu}")

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

recall = recall_score(y_selected, y_pred, pos_label=1) 
print(f"Recall (Good): {recall:.4f}")
precision = precision_score(y_selected, y_pred, pos_label=1)
print(f"Precision (Good): {precision:.4f}")
f1 = f1_score(y_selected, y_pred, pos_label=1)
print(f"F1-Score (Good): {f1:.4f}")
accuracy = accuracy_score(y_selected, y_pred)
print(f"Accuracy: {accuracy:.4f}")




In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns 

cm = confusion_matrix(y_selected, y_pred)
def plot_confusion_matrix(cm, class_names):
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] 
    plt.figure(figsize=(6, 4.5))
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('')
    plt.show()
class_names = ['Defective', 'Good']

plot_confusion_matrix(cm, class_names)


In [ ]:
import numpy as np
from sklearn.metrics import auc
import matplotlib.pyplot as plt


y_scores = oc_svm.decision_function(X_scaled) 
y_true = (y_selected != -1).astype(int)     

thresholds = np.linspace(min(y_scores), max(y_scores), 100)

tpr_list = []
fpr_list = []

for threshold in thresholds:
    y_pred = (y_scores >= threshold).astype(int)
    
    TP = np.sum((y_true == 1) & (y_pred == 1)) 
    FP = np.sum((y_true == 0) & (y_pred == 1))  
    FN = np.sum((y_true == 1) & (y_pred == 0)) 
    TN = np.sum((y_true == 0) & (y_pred == 0))  

    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0  
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  

    tpr_list.append(TPR)
    fpr_list.append(FPR)

roc_auc = auc(fpr_list, tpr_list)

plt.figure(figsize=(8, 8))
plt.plot(fpr_list, tpr_list, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')  # 対角線
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

print(f"AUC Score: {roc_auc:.4f}")
print(y_scores)


In [ ]:
import numpy as np
from sklearn.metrics import auc
import matplotlib.pyplot as plt

y_scores = oc_svm.decision_function(X_scaled) 
y_true = (y_selected == -1).astype(int) 
thresholds = np.linspace(min(y_scores), max(y_scores), 100)

tpr_list = []
fpr_list = []

for threshold in thresholds:
    y_pred = (y_scores <= threshold).astype(int)
    
    TP = np.sum((y_true == 1) & (y_pred == 1))  
    FP = np.sum((y_true == 0) & (y_pred == 1)) 
    FN = np.sum((y_true == 1) & (y_pred == 0))  
    TN = np.sum((y_true == 0) & (y_pred == 0)) 

    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0  
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  

    tpr_list.append(TPR)
    fpr_list.append(FPR)

roc_auc = auc(fpr_list, tpr_list)

plt.figure(figsize=(8, 8))
plt.plot(fpr_list, tpr_list, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--') 
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

print(f"AUC Score: {roc_auc:.4f}")
print(y_scores)
